In [1]:
%%time

import glob

import geopandas as gpd
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
from libpysal.graph import read_parquet
from sklearn.preprocessing import PowerTransformer, RobustScaler, StandardScaler

from fast_hdbscan.numba_kdtree import kdtree_to_numba
from fast_hdbscan.numba_kdtree import parallel_tree_query
from sklearn.neighbors import KDTree

CPU times: user 11 s, sys: 399 ms, total: 11.4 s
Wall time: 9.05 s


In [2]:
region_id = 69300

tessellations_dir = '/data/uscuni-ulce/processed_data/tessellations/'
chars_dir = "/data/uscuni-ulce/processed_data/chars/"
graph_dir = "/data/uscuni-ulce/processed_data/neigh_graphs/"

In [3]:
graph = read_parquet(graph_dir + f"tessellation_graph_{region_id}_knn1.parquet")

In [4]:
graph.cardinalities.describe()

count    304554.000000
mean          6.751085
std           2.060782
min           1.000000
25%           6.000000
50%           7.000000
75%           8.000000
max          82.000000
Name: cardinalities, dtype: float64

In [5]:
from core.cluster_validation import print_distance, generate_neigbhourhood_groups, generate_detailed_clusters

In [6]:
tessellation = gpd.read_parquet(
        tessellations_dir + f"tessellation_{region_id}.parquet"
)

In [90]:
X_train = pd.read_parquet(chars_dir + f'primary_chars_{region_id}.parquet')



X_train = X_train[X_train.index >= 0]



spatial_lag = 2


# lag = pd.read_parquet(f'/data/uscuni-ulce/processed_data/context_data/context_chars_{region_id}_lag_{spatial_lag}.parquet')

lag = pd.read_parquet(f'/data/uscuni-ulce/processed_data/context_data/unprocessed_context_chars_{region_id}_lag_{spatial_lag}.parquet')



# X_train = X_train.join(lag[[c for c in lag.columns if '_median' in c]], how='inner')
X_train = X_train.join(lag, how='inner')


In [91]:
# for c in X_train.columns:
#     X_train[c] = X_train[c].clip(*np.percentile(X_train[c], [5, 95]))

In [92]:
to_drop = ['stcSAl',
 'ltkOri',
 'stbOri',
 'stcOri',
 'stbCeA']


all_drop = []
for c in to_drop:
    all_drop += X_train.columns[X_train.columns.str.contains(c)].tolist()


X_train = X_train.drop(all_drop, axis=1)

In [93]:
vals = StandardScaler().fit_transform(X_train)
X_train = pd.DataFrame(vals, columns=X_train.columns, index=X_train.index)

vals = np.nan_to_num(X_train)
X_train = pd.DataFrame(vals, columns=X_train.columns, index=X_train.index)


# X_train = X_train.clip(-10, 10)

In [94]:
# t1 = X_train[[c for c in X_train.columns if '_' not in c]]
# t2 = X_train[[c for c in X_train.columns if '_median' in c]]

# X_train = t1.join(t2)
# X_train.shape

In [95]:
stats = X_train.describe()
stats.columns[stats.loc['std'] == 0]

Index(['sdbCoA_lower'], dtype='object')

In [96]:
X_train = X_train.drop(stats.columns[stats.loc['std'] == 0], axis=1)

In [97]:
X_train.shape

(299064, 231)

In [98]:
tess_groups = generate_detailed_clusters(tessellation,
                                         include_random_sample=False)
tess_groups = tess_groups[tess_groups.index.isin(X_train.index)]
tess_groups_ilocs = (
    pd.Series(np.arange(len(X_train)), index=X_train.index)
    .loc[tess_groups.index]
    .values
)

def check_score(data, example_clusters):
    groups = example_clusters[example_clusters.index.isin(data.index)]
    groups_ilocs = (
        pd.Series(np.arange(len(data)), index=data.index).loc[groups.index].values
    )
    return davies_bouldin_score(data.iloc[groups_ilocs], groups.values)

In [99]:
# tessellation.loc[tess_groups.index].explore()

In [100]:
from core.cluster_validation import print_distance
print_distance( pd.DataFrame(X_train.loc[tess_groups.index]).groupby(tess_groups.values).mean(), metric='euclidean')

,commie blocks vn,fancy commie blocks,holyne,housing blocks,housing houses,josefov,karlin IT offices,karlin old,karlin river offices,karlin square,mala strana,malesice,prague castle,row houses1,row houses2,smickov,stare mesto,vinohrady blocks,vinohrady squares,vinohrady villas
commie blocks vn,0.000000,10.302087,13.727224,11.501300,12.856407,16.996998,12.288429,19.189080,17.668311,12.653818,22.082182,15.277447,19.455400,19.025453,20.746765,15.378090,23.727604,11.203782,17.870093,13.345262
fancy commie blocks,10.302087,0.000000,14.666666,8.411398,13.752162,20.178610,10.915854,19.720883,13.222597,15.558795,22.888493,11.795653,17.172563,20.924339,22.123522,17.715691,25.442685,11.973185,19.626620,13.914831
holyne,13.727224,14.666666,0.000000,14.220885,8.860589,18.322276,13.575186,20.664780,19.385384,15.030034,22.114894,14.171401,18.418899,20.573795,20.780169,17.394745,24.560578,9.938496,19.163582,10.236458
housing blocks,11.501300,8.411398,14.220885,0.000000,12.353809,17.548206,11.258561,18.699249,12.693596,14.660122,21.350604,12.517778,15.271147,22.075927,23.284454,15.967605,23.209937,10.496971,19.040185,13.296639
housing houses,12.856407,13.752162,8.860589,12.353809,0.000000,15.302009,11.187415,18.272102,19.200729,11.752264,22.863466,14.927128,19.428174,18.851319,19.401866,13.840243,23.540426,8.724817,14.465300,5.074803
josefov,16.996998,20.178610,18.322276,17.548206,15.302009,0.000000,18.814576,13.887934,24.428794,9.412141,16.662119,21.659924,17.982680,22.760876,24.074290,7.810641,13.232138,16.416894,12.009832,15.317389
karlin IT offices,12.288429,10.915854,13.575186,11.258561,11.187415,18.814576,0.000000,18.177203,15.258281,13.726567,23.301094,9.985566,18.981479,19.185357,20.266916,15.726553,24.900326,8.913662,17.443710,11.241746
karlin old,19.189080,19.720883,20.664780,18.699249,18.272102,13.887934,18.177203,0.000000,24.136351,10.508643,15.101413,19.355727,16.324849,19.857619,21.007461,9.334537,13.816301,17.381086,11.684863,17.196702
karlin river offices,17.668311,13.222597,19.385384,12.693596,19.200729,24.428794,15.258281,24.136351,0.000000,21.133002,26.109689,14.285892,20.693683,26.553329,27.480494,22.267019,28.773053,14.973335,24.576566,19.187035
karlin square,12.653818,15.558795,15.030034,14.660122,11.752264,9.412141,13.726567,10.508643,21.133002,0.000000,18.135275,16.828972,17.100113,17.786797,19.271970,5.264933,17.055290,12.342672,8.306077,11.208771


In [101]:
from scipy.spatial.distance import pdist
for i, g in X_train.loc[tess_groups.index].groupby(tess_groups.values):
    print(i, np.mean(pdist(g)))

commie blocks vn 14.792654628015129
fancy commie blocks 14.728580327455491
holyne 14.754867217268467
housing blocks 17.055570015534563
housing houses 11.065896285894555
josefov 15.83788149733289
karlin IT offices 16.68630651524196
karlin old 18.202808190889783
karlin river offices 20.410559453810947
karlin square 17.570724669405887
mala strana 28.186107288248923
malesice 23.456606947517756
prague castle 27.78243302246649
row houses1 16.959159114101748
row houses2 14.519074212620694
smickov 17.516058576264783
stare mesto 23.909225910143036
vinohrady blocks 11.093138027996476
vinohrady squares 12.14431321174739
vinohrady villas 10.418134486039994


In [102]:
# tessellation.loc[tess_groups.index].explore(column=tess_groups.values, categorical=True)

In [103]:
mean_clusters = pd.DataFrame(X_train.loc[tess_groups.index]).groupby(tess_groups.values).mean()

In [104]:
(mean_clusters.loc['josefov'] - mean_clusters.loc['vinohrady squares']).abs().sort_values(ascending=False)

ssbSqu_median    3.012468
linP4W_lower     3.009500
linP4W_median    2.898961
linP4W           2.875372
linP3W_median    2.820783
                   ...   
ldkAre           0.003503
lskERI_higher    0.002026
sssLin_higher    0.000940
likWCe_higher    0.000143
sdbCoA_median    0.000000
Length: 231, dtype: float64

In [105]:
(mean_clusters.loc['josefov'] - mean_clusters.loc['stare mesto']).abs().sort_values(ascending=False)

libNCo_higher    5.761905
libNCo_median    5.015810
libNCo           4.866751
libNCo_lower     2.712671
ssbERI_median    2.356792
                   ...   
sdsLen_lower     0.002664
linP4W_higher    0.000958
midAre_median    0.000912
ssbCor           0.000406
likWCe_higher    0.000066
Length: 231, dtype: float64

In [106]:
from core.utils import used_keys
used_keys['ldbPWL']

'perimeter wall length of adjacent buildings'

In [107]:
# training_data = X_train.loc[tess_groups.index]
# tess_groups_ilocs = (
#     pd.Series(np.arange(len(training_data)), index=training_data.index)
#     .loc[tess_groups.index]
#     .values
# )
# training_data.shape

In [108]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import fcluster
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import davies_bouldin_score
from core.cluster_validation import get_linkage_matrix

In [109]:
q1 = read_parquet(graph_dir + f"tessellation_graph_{region_id}_knn1.parquet")

# clustering_graph = q1.higher_order(k=3, lower_order=True, diagonal=True).subgraph(X_train.index.values)

clustering_graph = q1.copy()

In [110]:
graph_labels = q1.subgraph(X_train.index.values).component_labels
graph_labels.value_counts()

component labels
776    60555
452    25177
754    17243
99     15583
726    13382
       ...  
247        1
727        1
249        1
250        1
161        1
Name: count, Length: 919, dtype: int64

In [111]:
clustering_graph = clustering_graph.subgraph(graph_labels[graph_labels == 776].index.values)

In [112]:
core_ids = clustering_graph.unique_ids
clustering_graph = clustering_graph.transform('B').sparse

In [113]:
training_data = X_train[X_train.index.isin(core_ids)]

In [114]:


# training_data = training_data[[c for c in training_data.columns if '_' not in c]]


In [115]:
# t1 = training_data[[c for c in training_data.columns if '_' not in c]]
# t2 = training_data[[c for c in training_data.columns if '_median' in c]]

# training_data = t1.join(t2)

In [116]:
training_data.shape

(60555, 231)

In [117]:
# training_data = training_data[[c for c in training_data.columns if '_' not in c]]
# training_data.shape

In [118]:
%%time
clusterer = AgglomerativeClustering(linkage='ward',
                                    connectivity=clustering_graph, 
                                    compute_full_tree=True, compute_distances=True)
model = clusterer.fit(training_data)

CPU times: user 1.68 s, sys: 60 ms, total: 1.74 s
Wall time: 1.73 s


In [119]:
linkage_matrix = get_linkage_matrix(model)

In [120]:
# fix, ax = plt.subplots(figsize=(40,40))
# # Plot the corresponding dendrogram
# _ = dendrogram(linkage_matrix, truncate_mode="level", p=5, ax=ax)

In [121]:
from sklearn.metrics import calinski_harabasz_score

In [122]:
tess_groups = generate_detailed_clusters(tessellation,
                                         include_random_sample=False)
tess_groups = tess_groups[tess_groups.index.isin(training_data.index)]
tess_groups_ilocs = (
    pd.Series(np.arange(len(training_data)), index=training_data.index)
    .loc[tess_groups.index]
    .values
)

In [123]:
for t in range(25, 600, 25):
    r = fcluster(linkage_matrix, t=t, criterion='distance')
    # r = pd.Series(r, index=X_train.index)
    # # ssplits = graph.describe(r, statistics=['nunique'])['nunique']
    print(t, ' - ', 
          adjusted_rand_score(tess_groups.values, r[tess_groups_ilocs]),
          # (ssplits > 1).sum() / ssplits.shape[0],
          davies_bouldin_score(training_data, r),
          calinski_harabasz_score(training_data, r)
         )

25  -  0.06089847894818099 1.6006992059244358 72.4822489896885
50  -  0.20127994730874443 2.1619906550200967 131.1524264038849
75  -  0.32994296088047 2.5735855952729376 195.70376001471098
100  -  0.4457162004341455 2.9056852232621777 262.71673066280067
125  -  0.5506976292489489 3.186854165874433 336.25154667151907
150  -  0.5751229934484416 3.4686283167693097 399.66331250159095
175  -  0.6378425843455878 3.8683595788408423 479.450038060521
200  -  0.6339800781025168 3.892929294212222 560.3202391265686
225  -  0.6495105127620552 3.934526386799841 669.071247181485
250  -  0.6464200950097665 4.107892909092207 755.9063384671437
275  -  0.65879247317748 4.0623518805604855 818.1328613987125
300  -  0.6055756794858271 3.7442898090229235 971.3813808157865
325  -  0.5151886383878395 3.7582356946678988 1027.4354164965362
350  -  0.4757305297803432 3.3433529233713806 1144.3425319420267
375  -  0.4757896407410364 3.831137628424043 1261.5170010637235
400  -  0.4117417047821117 4.50586950432712 14

In [124]:
# for t in range(5, 25, 1):
#     r = fcluster(linkage_matrix, t=t, criterion='distance')
#     # r = pd.Series(r, index=X_train.index)
#     # # ssplits = graph.describe(r, statistics=['nunique'])['nunique']
#     print(t, ' - ', 
#           adjusted_rand_score(tess_groups.values, r[tess_groups_ilocs]),
#           # (ssplits > 1).sum() / ssplits.shape[0],
#           davies_bouldin_score(training_data, r),
#           calinski_harabasz_score(training_data, r)
#          )

In [44]:
plotting = tessellation.loc[training_data.index].reset_index()

In [45]:
%%time
import lonboard
# plotting = tessellation[tessellation.index.isin(X_train.index)].copy()
layer = lonboard.PolygonLayer.from_geopandas(plotting, opacity=.08)

/home/krasen/miniconda3/envs/urban_taxonomy/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


CPU times: user 1.14 s, sys: 91.8 ms, total: 1.24 s
Wall time: 1.23 s


In [46]:
from sidecar import Sidecar
sc = Sidecar(title='Clusters')
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.Positron)
with sc:
    display(m)

In [47]:
from core.cluster_validation import get_color

In [127]:
clusters = fcluster(linkage_matrix, t=200, criterion='distance')
np.unique(clusters, return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
        52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
        69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
        86], dtype=int32),
 array([  72,  197,  263,   34,    1, 1438, 1128,  706,   69,  211,  187,
        1090,  219, 4687,  980,  254,  353, 1769,  781, 2544,  274,  368,
         309,  521,  557, 1133, 1082,  364,  498,  467,  925, 1157, 2398,
        1169,  149,  625,  613,  215,  188,  289,  814,   48,  716,  706,
        1348,  601,  243, 1242,  507, 1270,  762,   35,   11,  258,  195,
        1713,  707, 1595,  592, 2282, 2176,  218,  632,  512,  547,  901,
        1052,    1,   24,  195,  243,  319,   18,   93, 1353,  139,   80,
        3360,   32,  168, 1246,  570,  385,   19,  342,    1]))

In [128]:
layer.get_fill_color = get_color(clusters)

In [ ]:
karlin_river_offices = [
    282517, 282519, 282516, 282503, 282509, 282513, 282511, 282514, 261760, 261761, 261762
]


In [157]:
from core.cluster_validation import get_feature_importance
from core.utils import used_keys

In [69]:
clusters_subset = [11597, 11615, 17742]
clusters_subset = np.where(np.isin(clusters, clusters[clusters_subset]))

In [70]:
imps = get_feature_importance(training_data.iloc[clusters_subset], clusters[clusters_subset])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished


In [72]:
imps.loc[:10, [c for c in imps.columns if '_vals' not in c]]

,cluster_48,cluster_66,cluster_69
0,linWID_higher,lcdMes_higher,ssbCCD_lower
1,ldsMSL_lower,linPDE,ltkOri_lower
2,lddNDe,linP3W_higher,ssbSqu_lower
3,ltkOri_lower,mtbAli_median,lcdMes_median
4,ldsMSL,lskCWA_lower,linWID_higher
5,ldsMSL_median,lskERI_median,sicCAR_lower
6,midRea_median,ltkWNB_median,lskCWA_lower
7,ldkAre_higher,linWID_lower,stcOri_lower
8,lskCCo_lower,mtbNDi_higher,ssbSqu_median
9,ssbCCD_lower,linPDE_lower,sdcLAL_median


In [77]:
used_keys['ltkOri']

'orientation of enclosure'

In [112]:
imps = get_feature_importance(training_data, clusters)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 co

In [119]:
imps.loc[:10, [c for c in imps.columns if '_vals' not in c]]

,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7
0,ldkPer,lcdMes,libNCo,lskCWA,ldsAre,ltkWNB,likWBB
1,ltkWNB,likWBB,likWBB,likWBB,midAre,ldkPer,lcdMes
2,lskCCo,ltkWNB,sdsSPO,ltkWNB,ltkOri,lskCCo,ltkWNB
3,ltkOri,lskERI,ldbPWL,lskCCo,linP4W,lskCWA,linWID
4,lskCWA,lskCWA,ltcBuA,ldkAre,sdsSPO,ltkOri,ltkOri
5,ldkAre,ldkAre,sicCAR,lskERI,ldsMSL,likWBB,lskCWA
6,likWBB,linP4W,mtbSWR,ldsAre,lddNDe,ldkAre,ldkAre
7,lcdMes,ldkPer,ssbSqu,lddNDe,lcdMes,lskERI,lddNDe
8,lskERI,ltkOri,lskERI,midRea,ldkPer,linP3W,linP3W
9,linP4W,linPDE,sdbPer,linPDE,likWBB,ldsAre,lcnClo


In [122]:
used_keys['lcdMes']

'local meshedness of street network'

In [ ]:
imps[[c for c in imps.columns if '_vals' in c]].cumsum(axis=1)

In [96]:
josefov_joins = []

josefov_joins.append(np.isin(linkage_matrix[:, 0], 
                             tess_groups_ilocs[tess_groups == 'josefov']))
josefov_joins.append(np.isin(linkage_matrix[:, 1], 
                             tess_groups_ilocs[tess_groups == 'josefov']))


In [97]:
indxs = linkage_matrix[josefov_joins[0] | josefov_joins[1]]
indxs = np.union1d(indxs[:, 0], indxs[:, 1])
indxs = indxs[indxs <= X_train.shape[0]]

In [150]:
indxs = linkage_matrix[linkage_matrix[:, 2] <= 2]
indxs = np.union1d(indxs[:, 0], indxs[:, 1])
indxs = indxs[indxs < X_train.shape[0]]
indxs.shape

(64349,)

In [151]:
plotting = tessellation.loc[X_train.iloc[indxs].index]

In [42]:
cluster_means = training_data.groupby(clusters).mean()

In [68]:
c1 = 6
c2 = 10

(cluster_means.loc[c1] - cluster_means.loc[c2]).sort_values(ascending=False)

libNCo           6.284668
libNCo_median    5.846288
libNCo_higher    4.233946
linPDE_higher    3.014914
linPDE           2.122206
                   ...   
lcnClo          -1.414220
linWID_lower    -1.422183
linP3W          -1.489277
lcnClo_lower    -1.702326
linP3W_lower    -2.339139
Length: 248, dtype: float64

In [72]:
from core.utils import used_keys
used_keys['libNCo']

'number of courtyards within adjacent buildings'

In [169]:
bgraph = read_parquet(graph_dir + f"building_graph_{region_id}_knn1.parquet")

In [170]:
buildings_dir = '/data/uscuni-ulce/processed_data/buildings/'

buildings = gpd.read_parquet(
        buildings_dir + f"buildings_{region_id}.parquet"

)

In [178]:
buildings

,index,id,geometry
0,0,v0.1-CZE.12.2_1-35164,"POLYGON ((4614847.626 2975218.938, 4614848.235..."
1,1,v0.1-CZE.12.2_1-35123,"POLYGON ((4615276.357 2976034.184, 4615282.866..."
2,2,v0.1-CZE.12.2_1-35159,"POLYGON ((4615315.503 2975986.2, 4615322.056 2..."
3,3,v0.1-CZE.12.2_1-35166,"POLYGON ((4615222.339 2976016.91, 4615224.582 ..."
4,4,v0.1-CZE.12.2_1-35228,"POLYGON ((4615300.348 2975924.258, 4615301.6 2..."
...,...,...,...
299059,299060,v0.1-CZE.13.3_1-13696,"POLYGON ((4618611.169 3033535.197, 4618623.01 ..."
299060,299061,v0.1-CZE.13.3_1-13674,"POLYGON ((4618611.989 3033568.153, 4618617.119..."
299061,299062,v0.1-CZE.13.3_1-13591,"POLYGON ((4618614.831 3033550.704, 4618628.289..."
299062,299063,v0.1-CZE.13.3_1-13328,"POLYGON ((4618625.628 3033512.926, 4618625.634..."


In [181]:
buildings = buildings.join(X_train, how='inner').drop(['index', 'id'], axis=1)

In [185]:
r = buildings.dissolve(bgraph.component_labels, aggfunc='mean')

In [189]:
plotting = r

In [190]:
%%time
import lonboard
# plotting = tessellation[tessellation.index.isin(X_train.index)].copy()
layer = lonboard.PolygonLayer.from_geopandas(plotting, opacity=.08)

/home/krasen/miniconda3/envs/urban_taxonomy/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


CPU times: user 1.65 s, sys: 178 ms, total: 1.83 s
Wall time: 1.82 s


In [191]:
from sidecar import Sidecar
sc = Sidecar(title='Clusters')
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.Positron)
with sc:
    display(m)

In [156]:
enclosures = gpd.read_parquet(f"/data/uscuni-ulce/processed_data/enclosures/enclosure_{region_id}.parquet")
encl_counts = tessellation.groupby('enclosure_index').count()
encl_counts.columns = ['tessellation']
enclosures['lieWCe'] = encl_counts['tessellation'] / enclosures.geometry.area

7        0.000325
8        0.000224
11       0.000030
12       0.000122
14       0.000638
           ...   
25157    0.000100
25158    0.000029
25159    0.000101
25160    0.000399
25161    0.000081
Length: 15958, dtype: float64

In [251]:
enclosures['lieWCe'] 

7        0.000325
8        0.000224
11       0.000030
12       0.000122
14       0.000638
           ...   
25157    0.000100
25158    0.000029
25159    0.000101
25160    0.000399
25161    0.000081
Name: lieWCe, Length: 15958, dtype: float64

In [164]:
encl_counts['tessellation']

enclosure_index
7        199
8         52
11         1
12        25
14       962
        ... 
25157      9
25158      1
25159     13
25160     15
25161      3
Name: tessellation, Length: 15935, dtype: int64

In [ ]:
# Measure weighted cells within enclosure
merged = enclosures[['eID', 'ldeAre']].merge(encl_counts[['tessellation']], how='left', on='eID')
enclosures['lieWCe'] = merged['tessellation'] / merged['ldeAre']